In [8]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd

In [9]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,11 mins 09 secs
H2O_cluster_timezone:,America/Sao_Paulo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,20 days
H2O_cluster_name:,H2O_from_python_luizbueno_uzspgo
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.987 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [14]:
#Importa dados 
imp = pd.read_csv("Dados/Churn_treino.csv", sep = ";")

In [15]:
imp

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0,1,1,1,10134888,1
1,608,Spain,Female,41,1,8380786,1,0,1,11254258,0
2,502,France,Female,42,8,1596608,3,1,0,11393157,1
3,699,France,Female,39,1,0,2,0,0,9382663,0
4,850,Spain,Female,43,2,12551082,1,1,1,790841,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0,2,1,0,9627064,0
9996,516,France,Male,35,10,5736961,1,1,1,10169977,0
9997,709,France,Female,36,7,0,1,0,1,4208558,1
9998,772,Germany,Male,42,3,7507531,2,1,0,9288852,1


In [16]:
# Tranformando o formato do pandas para o formato do h2o
imp = h2o.H2OFrame(imp)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [18]:
# Dividir os dados em treino e teste
train, test = imp.split_frame(ratios=[0.7])

In [20]:
test.shape, train.shape

((3017, 11), (6983, 11))

In [23]:
# Transformando nossa váriavel de resposta em fator

train["Exited"] = train["Exited"].asfactor()
test["Exited"] = test["Exited"].asfactor()

In [25]:
# criação do objeto do modelo
# Vamos definir o tempo máximo de processamento em 60 segundos, apenas para fins didáticos, é um tempo pequeno, poderiamos limitar também pelo número de modelos, estipular um número máximo de modelos.


modelo = H2OAutoML(max_runtime_secs=60)

In [26]:
modelo

In [27]:
# treinamento do modelo
# Utilizamos a variável exited para ver se o Funcionário abandonou ou não a empresa.

modelo.train(y="Exited", training_frame=train)

AutoML progress: |
11:38:06.805: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_2_AutoML_1_20220504_113806

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.07517433824010908
RMSE: 0.2741793906188229
LogLoss: 0.2543279278458461
Null degrees of freedom: 6982
Residual degrees of freedom: 6975
Null deviance: 7074.801968987091
Residual deviance: 3551.9438402950877
AIC: 3567.9438402950877
AUC: 0.9379747848734454
AUCPR: 0.8400787270807851
Gini: 0.8759495697468909

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.32787707542906114: 


,,0,1,Error,Rate
0,0,5194.0,361.0,0.065,(361.0/5555.0)
1,1,370.0,1058.0,0.2591,(370.0/1428.0)
2,Total,5564.0,1419.0,0.1047,(731.0/6983.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.327877,0.743238,195.0
1,max f2,0.174297,0.796648,262.0
2,max f0point5,0.511568,0.795107,137.0
3,max accuracy,0.469314,0.899327,150.0
4,max precision,0.992975,1.000000,0.0
5,max recall,0.024840,1.000000,381.0
6,max specificity,0.992975,1.000000,0.0
7,max absolute_mcc,0.362308,0.679868,184.0
8,max min_per_class_accuracy,0.205439,0.857066,246.0
9,max mean_per_class_accuracy,0.210818,0.858954,243.0



Gains/Lift Table: Avg response rate: 20,45 %, avg score: 20,62 %


...


Gains/Lift Table: Avg response rate: 20,45 %, avg score: 20,46 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010024,0.948428,4.820198,4.820198,0.985714,0.964711,0.985714,0.964711,0.048319,0.048319,382.019808,382.019808,0.048139
1,2,0.020049,0.920166,4.750340,4.785269,0.971429,0.931601,0.978571,0.948156,0.047619,0.095938,375.034014,378.526911,0.095398
2,3,0.030073,0.897241,4.401050,4.657196,0.900000,0.909404,0.952381,0.935239,0.044118,0.140056,340.105042,365.719621,0.138256
3,4,0.040097,0.868784,4.470908,4.610624,0.914286,0.881904,0.942857,0.921905,0.044818,0.184874,347.090836,361.062425,0.181994
4,5,0.050122,0.837732,4.470908,4.582681,0.914286,0.853631,0.937143,0.908250,0.044818,0.229692,347.090836,358.268107,0.225731
5,6,0.100100,0.628108,3.530929,4.057557,0.722063,0.737827,0.829757,0.823161,0.176471,0.406162,253.092870,305.755721,0.384740
6,7,0.150079,0.450628,2.550115,3.555556,0.521490,0.534925,0.727099,0.727174,0.127451,0.533613,155.011518,255.555600,0.482128
7,8,0.200057,0.328634,1.681395,3.087351,0.343840,0.383115,0.631353,0.641221,0.084034,0.617647,68.139462,208.735105,0.524938
8,9,0.300014,0.199413,1.324098,2.499881,0.270774,0.255588,0.511217,0.512738,0.132353,0.750000,32.409826,149.988067,0.565662
9,10,0.399971,0.133227,0.735610,2.058971,0.150430,0.163887,0.421053,0.425556,0.073529,0.823529,-26.438985,105.897096,0.532440




Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.852166,0.022359,0.868185,0.847357,0.817664,0.875176,0.852447
1,auc,0.862847,0.010609,0.873647,0.861003,0.855585,0.873748,0.850253
2,err,0.147834,0.022359,0.131815,0.152643,0.182336,0.124824,0.147553
3,err_count,206.200000,29.928247,191.000000,205.000000,256.000000,177.000000,202.000000
4,f0point5,0.645270,0.055982,0.665354,0.623209,0.577742,0.728346,0.631700
5,f1,0.638788,0.022045,0.638941,0.629295,0.628985,0.676417,0.620301
6,f2,0.636191,0.032141,0.614545,0.635500,0.690204,0.631399,0.609306
7,lift_top_group,4.834543,0.313094,5.138298,4.937500,4.775510,4.325055,4.996350
8,logloss,0.336386,0.012224,0.319702,0.338838,0.352211,0.330037,0.341139
9,max_per_class_error,0.363229,0.058975,0.400709,0.360294,0.261905,0.395425,0.397810



See the whole table with table.as_data_frame()


In [28]:
ranking = modelo.leaderboard

In [29]:
ranking

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_2_AutoML_1_20220504_113806,0.862918,0.33622,0.702931,0.238453,0.319254,0.101923
StackedEnsemble_BestOfFamily_4_AutoML_1_20220504_113806,0.862655,0.336114,0.704229,0.238564,0.319289,0.101945
StackedEnsemble_AllModels_3_AutoML_1_20220504_113806,0.862137,0.336702,0.702758,0.237091,0.319605,0.102148
GBM_grid_1_AutoML_1_20220504_113806_model_11,0.861737,0.337723,0.701766,0.228766,0.319846,0.102301
StackedEnsemble_BestOfFamily_3_AutoML_1_20220504_113806,0.861262,0.338031,0.699241,0.229716,0.320018,0.102412
GBM_grid_1_AutoML_1_20220504_113806_model_8,0.860927,0.339621,0.699879,0.23457,0.320599,0.102784
StackedEnsemble_AllModels_1_AutoML_1_20220504_113806,0.8606,0.338613,0.699929,0.236012,0.320162,0.102504
GBM_5_AutoML_1_20220504_113806,0.860244,0.339252,0.698582,0.241414,0.320573,0.102767
GBM_grid_1_AutoML_1_20220504_113806_model_1,0.859834,0.341238,0.696248,0.232578,0.321629,0.103445
StackedEnsemble_BestOfFamily_2_AutoML_1_20220504_113806,0.859035,0.340404,0.694404,0.237071,0.321088,0.103097


In [30]:
ranking = ranking.as_data_frame()

In [31]:
ranking

,model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
0,StackedEnsemble_AllModels_2_AutoML_1_20220504_113806,0.862918,0.336220,0.702931,0.238453,0.319254,0.101923
1,StackedEnsemble_BestOfFamily_4_AutoML_1_20220504_113806,0.862655,0.336114,0.704229,0.238564,0.319289,0.101945
2,StackedEnsemble_AllModels_3_AutoML_1_20220504_113806,0.862137,0.336702,0.702758,0.237091,0.319605,0.102148
3,GBM_grid_1_AutoML_1_20220504_113806_model_11,0.861737,0.337723,0.701766,0.228766,0.319846,0.102301
4,StackedEnsemble_BestOfFamily_3_AutoML_1_20220504_113806,0.861262,0.338031,0.699241,0.229716,0.320018,0.102412
5,GBM_grid_1_AutoML_1_20220504_113806_model_8,0.860927,0.339621,0.699879,0.234570,0.320599,0.102784
6,StackedEnsemble_AllModels_1_AutoML_1_20220504_113806,0.860600,0.338613,0.699929,0.236012,0.320162,0.102504
7,GBM_5_AutoML_1_20220504_113806,0.860244,0.339252,0.698582,0.241414,0.320573,0.102767
8,GBM_grid_1_AutoML_1_20220504_113806_model_1,0.859834,0.341238,0.696248,0.232578,0.321629,0.103445
9,StackedEnsemble_BestOfFamily_2_AutoML_1_20220504_113806,0.859035,0.340404,0.694404,0.237071,0.321088,0.103097


In [32]:
# Agora iremos realizar nossa previsão:

test = pd.read_csv("Dados/Churn_prever.csv", sep = ";")
test = h2o.H2OFrame(test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [35]:
forecast = modelo.leader.predict(test)   # aqui nós escolhemos o melhor modelo.

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [39]:
# Tranformando a previsão  num dataframe
forecast = forecast.as_data_frame(forecast)

AttributeError: 'DataFrame' object has no attribute 'as_data_frame'

In [40]:
forecast  # onde possui 1 é possibilidade do funcionário deixar e empresa.

,predict,p0,p1
0,0,0.926101,0.073899
1,0,0.958882,0.041118
2,1,0.007838,0.992162
3,0,0.982560,0.017440
4,0,0.922025,0.077975
5,0,0.902415,0.097585
6,0,0.849196,0.150804
7,0,0.867086,0.132914
8,0,0.886442,0.113558
